<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Generalised_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install firebase-admin
import os 
import math
import cv2
import numpy as np 
import csv
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor


In [0]:
#Get Images from Firebase Storage
import datetime
import firebase_admin
from firebase_admin import credentials
from firebase_admin import storage

# Fetch the service account key JSON file contents
cred = credentials.Certificate("credentials.json")

# Initialize the app with a service account, granting admin privileges
app = firebase_admin.initialize_app(cred, {
    'storageBucket': 'fir-4ca2c.appspot.com',
}, name='storage')

In [0]:
#Loading E1 - Lenovo data : 504 -- sscb labels 

data = []
reading = []
t = 0
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imglenovo = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="lenovoinS" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
      blob.download_to_filename('/content/' + "lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')
      imglenovo.append("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg') 
      print("lenovo" + str(blob.name.split('/')[-1])+ '.jpeg')



In [0]:
print(imglenovo[0])
im_trans=[]
im_ent=[]
im_cont=[]
k=0
for img in imglenovo:
  im = cv2.imread(img)
  im = cv2.resize(im, (32,32))
  im_trans.append(np.ndarray.flatten(transmission(im, retMean = False )))
  im_ent.append(entropy(im))
  im_cont.append(contrast(im))
  k+=1
  print(k)
  
  
print(len(im_trans))
print(len(im_ent))
print(len(im_cont))

In [0]:
dict={}
for i in range(0,505):
  l=[]
  l.append(im_trans[i])
  l.append(im_ent[i])
  l.append(im_cont[i])
  dict[(imglenovo[i])]=l
  

In [0]:
##Loading E2 - Mi data : 760 -- sscb labels 
bucket = storage.bucket(app=app)
blobs = bucket.list_blobs()

imgminote = []

mits = set()
for blob in blobs:
  if blob.name[7:16]=="minoteSSC" :
    if int(blob.name[-8:-6])<19 and int(blob.name[-8:-6])>5:
        print(blob.name)

        blob.download_to_filename('/content/' + "minote" + str(blob.name.split('/')[-1])+ '.jpeg')
        imgminote.append("minote" + str(blob.name.split('/')[-1])+ '.jpeg') 
        print("minote" + str(blob.name.split('/')[-1])+ '.jpeg')


In [0]:
print(imgminote[0])

for img in imgminote:
  im = cv2.imread(img)
  im = cv2.resize(im, (32,32))
  im_trans.append(np.ndarray.flatten(transmission(im, retMean = False )))
  im_ent.append(entropy(im))
  im_cont.append(contrast(im))
  k+=1
  print(k)
  
  
print(len(im_trans))
print(len(im_ent))
print(len(im_cont))
  

In [0]:
k=0
for i in range(506,1330):
  l=[]
  l.append(im_trans[i])
  l.append(im_ent[i])
  l.append(im_cont[i])
  dict[(imgminote[k])]=l
  k+=1
  


In [82]:
k=0
for key in dict:
  k+=1
  if(k==505):
    print(key[:-5])
    print(dict[key])
  if(k==506):
    print(key)
    print(dict[key])
 

lenovo2019-07-02 14:43:53
[array([0.46902655, 0.46902655, 0.46902655, ..., 0.98672566, 0.98672566,
       0.98672566]), 33.49928283691406, 55.60050763749207]
minote2019-06-30 18:59:57.jpeg
[array([0.32941176, 0.32941176, 0.31764706, ..., 0.83921569, 0.92941176,
       0.92941176]), 43.589691162109375, 68.69164026918482]


In [195]:
#converting all image timestamps into cpcb compatible timestamp 
tmp=[]
for key in dict:
  temp=key[-24:-5]
  temp=temp.split('-')
  
  newtemp=temp[2].split(' ')
  
  newesttemp=newtemp[1].split(':')
  j=int(newesttemp[1])
  #print(j)
  if(j<=7.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  elif(j<=15 or j<=22.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '15'
  elif(j<=30 or j<=37.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '30'
  elif(j<=45 or j<=52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '45'
  elif(j>52.5):
    final = newtemp[0] + '-' + temp[1] + '-' + temp[0] + ' ' + newesttemp[0] + ':' + '00'
  tmp.append(final)
  
  
print(tmp)

['27-06-2019 17:00', '27-06-2019 17:00', '27-06-2019 17:15', '27-06-2019 17:15', '27-06-2019 18:00', '27-06-2019 18:15', '27-06-2019 18:15', '27-06-2019 18:15', '27-06-2019 18:30', '27-06-2019 18:30', '27-06-2019 18:30', '27-06-2019 18:45', '27-06-2019 18:45', '27-06-2019 18:45', '27-06-2019 18:00', '28-06-2019 06:00', '28-06-2019 06:00', '28-06-2019 06:15', '28-06-2019 06:15', '28-06-2019 06:15', '28-06-2019 06:30', '28-06-2019 06:30', '28-06-2019 06:30', '28-06-2019 06:45', '28-06-2019 06:45', '28-06-2019 06:45', '28-06-2019 06:00', '28-06-2019 07:00', '28-06-2019 07:00', '28-06-2019 07:15', '28-06-2019 07:15', '28-06-2019 07:15', '28-06-2019 07:30', '28-06-2019 07:30', '28-06-2019 07:30', '28-06-2019 07:45', '28-06-2019 07:45', '28-06-2019 07:45', '28-06-2019 07:00', '28-06-2019 08:00', '28-06-2019 08:00', '28-06-2019 08:15', '28-06-2019 08:15', '28-06-2019 08:15', '28-06-2019 08:30', '28-06-2019 08:30', '28-06-2019 08:30', '28-06-2019 08:45', '28-06-2019 08:45', '28-06-2019 08:45',

In [194]:
k=0
cpcb=[]
with open('/content/sscb_labels_27_2.csv') as file:
  reader = csv.reader(file, delimiter =',')
  for row in reader:
    k+=1
    if(k>=18):
      l=[]
      l.append(row[0])
      l.append(row[2])
      cpcb.append(l)      
      
print(len(cpcb))
print(cpcb)


466
[['27-06-2019 18:00', '36'], ['27-06-2019 18:15', '36'], ['27-06-2019 18:30', '36'], ['27-06-2019 18:45', '36'], ['27-06-2019 19:00', '42'], ['27-06-2019 19:15', '42'], ['27-06-2019 19:30', '42'], ['27-06-2019 19:45', '42'], ['27-06-2019 20:00', '59'], ['27-06-2019 20:15', '59'], ['27-06-2019 20:30', '59'], ['27-06-2019 20:45', '59'], ['27-06-2019 21:00', '74'], ['27-06-2019 21:15', '74'], ['27-06-2019 21:30', '74'], ['27-06-2019 21:45', '74'], ['27-06-2019 22:00', '59'], ['27-06-2019 22:15', '59'], ['27-06-2019 22:30', '59'], ['27-06-2019 22:45', '59'], ['27-06-2019 23:00', '67'], ['27-06-2019 23:15', '67'], ['27-06-2019 23:30', '67'], ['27-06-2019 23:45', '67'], ['28-06-2019 00:00', '85'], ['28-06-2019 00:15', '85'], ['28-06-2019 00:30', '85'], ['28-06-2019 00:45', '85'], ['28-06-2019 01:00', '94'], ['28-06-2019 01:15', '94'], ['28-06-2019 01:30', '94'], ['28-06-2019 01:45', '94'], ['28-06-2019 02:00', '117'], ['28-06-2019 02:15', '117'], ['28-06-2019 02:30', '117'], ['28-06-2019

In [149]:
print(ecttime[0][1])

[array([0.99607843, 0.99607843, 0.99607843, ..., 0.51764706, 0.52156863,
       0.52156863]), 33.62948989868164, 71.19818991492596]


In [0]:
final=[]
for e in ecttime:
  for tmp in cpcb:
    if(tmp[0] == e[0]):
      l=[]
      l.append(tmp[0])
      l.append(tmp[1])
      l.append(e[1])
      final.append(l)


In [155]:
print(len(final))

1239


In [163]:
print(final[0])

['27-06-2019 18:00', '36', [array([0.59215686, 0.59215686, 0.55294118, ..., 0.99607843, 0.99607843,
       0.99607843]), 35.64873504638672, 74.43500256452609]]


In [0]:
#Feature engineering
hours=[]
trans=[]
cont=[]
ent=[]

pm=[]

for e in final:
  hours.append(int(e[0][-5:-3]))
  if(e[1]=='None'):
    e[1]='NAN'
  pm.append(float(e[1]))
  trans.append(e[2][0])
  ent.append(e[2][1])
  cont.append(e[2][2])


In [0]:
print(len(hours))
print(hours)
print(len(trans))
print(trans)
print(len(cont))
print(cont)
print(len(ent))
print(ent)
print(len(pm))
print(pm)

In [174]:
print(trans)
print(np.shape(trans))

[0.59215686 0.59215686 0.55294118 ... 0.99607843 0.99607843 0.99607843]
(1239, 1024)


In [188]:
hours = np.reshape(hours,(1239,1))
trans = np.reshape(trans,(1239,1024))
cont = np.reshape(cont,(1239,1))
ent = np.reshape(ent,(1239,1))
pm = np.reshape(pm,(1239,1))

features = np.hstack((hours, cont, ent, trans))
print(features)

[[18.         74.43500256 35.64873505 ...  0.99607843  0.99607843
   0.99607843]
 [18.         74.03217162 35.67634201 ...  0.99607843  0.99607843
   0.99607843]
 [18.         74.30180835 35.45424271 ...  0.99607843  0.99607843
   0.99607843]
 ...
 [18.         62.20386814 45.96022034 ...  0.67088608  0.88607595
   0.88607595]
 [18.         61.75438578 46.32469177 ...  0.7257384   0.87763713
   0.87763713]
 [18.         60.90752465 47.71579742 ...  0.75        0.89583333
   0.89583333]]


In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
 
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont